<a href="https://colab.research.google.com/github/Dabinishere/Object-detection-optimization-study-of-deformed-images/blob/main/code/denoising/Denoising_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 디노이징

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


## NAFnet 이미지 넣기

### Git clone

In [ ]:
!git clone https://github.com/megvii-research/NAFNet
%cd NAFNet

Cloning into 'NAFNet'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 517 (delta 107), reused 88 (delta 65), pack-reused 360
Receiving objects: 100% (517/517), 16.20 MiB | 12.28 MiB/s, done.
Resolving deltas: 100% (255/255), done.
/content/NAFNet


### 환경 설정

In [ ]:
!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 36.1 MB/s 
     |████████████████████████████████| 190 kB 59.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14840 sha256=b521b6baca8d1bca40c9bba0282276f299d5f05265729a2f7e7d5cd7d070a9b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-7s0aw59y/wheels/94/8d/0b/bdcd83555c3555f91a33f6c2384428d9f163c7d75ab0d272b4
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
running develop
running egg_info
creating basicsr.egg-info
writing basicsr.egg-info/PKG-INFO
writing dependency_l

### pretrained 모델

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR', "./experiments/pretrained_models/", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR
To: /content/NAFNet/experiments/pretrained_models/NAFNet-SIDD-width64.pth
100%|██████████| 464M/464M [00:02<00:00, 208MB/s]


'./experiments/pretrained_models/NAFNet-SIDD-width64.pth'

### 준비

In [ ]:
import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)


### 기본 모델

In [ ]:
opt_path = 'options/test/SIDD/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

 load net keys <built-in method keys of collections.OrderedDict object at 0x7f1d88756d40>


### inference

In [ ]:
import glob
for i in range(1, 399):
  input_path = ('/content/drive/MyDrive/dataset(1)/image (%d).png'%(i))
  img_input = imread(input_path)
  inp = img2tensor(img_input)
  single_image_inference(NAFNet, inp, '/content/drive/MyDrive/dedata/image (%d).png'%(i))

### YOLOv5 이미지 넣기

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5x')  # or yolov5n - yolov5x6, custom


for i in range(1, 399):
  img=('/content/drive/MyDrive/dedata/image (%d).png'%(i))
  results=model(img)
  results.save(save_dir='/content/drive/MyDrive/resultpic/denoising_result', exist_ok=True)

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirements "gitpython" "ipython" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.de

  0%|          | 0.00/166M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/drive/MyDrive/resultpic/denoising_result
Saved 1 image to /content/d